In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P10', 'P11', 'P12', 'P13', 'P14']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp3.csv', index = True, header=True)

2020-04-27 16:27:08 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P10.tsv' as 'real location' with error 0.01
2020-04-27 16:27:08 INFO     Detected encoding: ascii
2020-04-27 16:27:08 INFO     Found 3200 rows and 2 columns
2020-04-27 16:27:08 DEBUG    Checking column names
2020-04-27 16:27:08 DEBUG    Index name 'index'
2020-04-27 16:27:08 DEBUG    Column name 'P10'
2020-04-27 16:27:08 INFO     Checking data format
2020-04-27 16:27:08 INFO     Column 'P10'
2020-04-27 16:27:08 INFO     count    3200.000000
2020-04-27 16:27:08 INFO     mean       -0.012175
2020-04-27 16:27:08 INFO     std         2.500738
2020-04-27 16:27:08 INFO     min       -42.287518
2020-04-27 16:27:08 INFO     50%         0.174727
2020-04-27 16:27:08 INFO     max         8.556823
2020-04-27 16:27:08 INFO     ---
2020-04-27 16:27:08 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with err

Time: 428 sec.

2020-04-27 16:34:17 INFO     Extracting autoclass results
2020-04-27 16:34:17 INFO     Found 3200 cases classified in 27 classes
2020-04-27 16:34:23 INFO     Aggregating input data
2020-04-27 16:34:23 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:34:23 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P11.tsv' as 'real location' with error 0.01
2020-04-27 16:34:23 INFO     Detected encoding: ascii
2020-04-27 16:34:23 INFO     Found 3200 rows and 2 columns
2020-04-27 16:34:23 DEBUG    Checking column names
2020-04-27 16:34:23 DEBUG    Index name 'index'
2020-04-27 16:34:23 DEBUG    Column name 'P11'
2020-04-27 16:34:23 INFO     Checking data format
2020-04-27 16:34:23 INFO     Column 'P11'
2020-04-27 16:34:23 INFO     count    3200.000000
2020-04-27 16:34:23 INFO     mean       -0.187907
2020-04-27 16:34:23 INFO     std         1.310756
2020-04-27 16:34:23 INFO     min       -11.829763
2020-04-27 16:34:23 INFO     50%   

[0.18518518518518517, 0.08143322475570032, 0.08794788273615635, 0.1018867924528302, 0.08243727598566308, 0.09239130434782608, 0.0891089108910891, 0.08235294117647059, 0.3218390804597701, 1.0, 0.08653846153846154, 0.11538461538461539, 0.15294117647058825, 0.16666666666666666, 0.1, 0.1694915254237288, 0.3333333333333333, 0.19696969696969696, 0.2033898305084746, 0.14285714285714285, 0.15, 1.0, 0.3142857142857143, 0.30303030303030304, 0.23529411764705882, 0.2, 0.4]
['CONTENT_FLASH', 'APPLE_ICLOUD', 'APPLE_ICLOUD', 'CLOUDFLARE', 'EBAY', 'HTTP_PROXY', 'INSTAGRAM', 'WINDOWS_UPDATE', 'DROPBOX', 'IP_ICMP', 'GOOGLE', 'APPLE_ITUNES', 'SPOTIFY', 'MS_ONE_DRIVE', 'CONTENT_FLASH', 'DNS', 'OFFICE_365', 'DROPBOX', 'HTTP', 'HTTP', 'SPOTIFY', 'DNS', 'WHATSAPP', 'FACEBOOK', 'MSN', 'REST_APP', 'APPLE_ICLOUD']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
0.23684315489283242


2020-04-27 16:34:23 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:34:23 INFO     Writing run file
2020-04-27 16:34:23 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:34:23 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:34:23 INFO     Running clustering...


...

[0.07709750566893424, 0.15547703180212014, 0.13127413127413126, 0.2288135593220339, 0.08, 0.09178743961352658, 0.10606060606060606, 0.08695652173913043, 0.0967741935483871, 0.0896551724137931, 0.13131313131313133, 1.0, 0.08333333333333333, 0.11235955056179775, 0.20754716981132076, 0.2597402597402597, 0.15625, 0.11320754716981132, 0.24074074074074073, 0.14814814814814814, 0.11764705882352941, 1.0, 0.391304347826087, 0.2857142857142857, 0.11764705882352941]
['MS_ONE_DRIVE', 'DROPBOX', 'AMAZON', 'CONTENT_FLASH', 'EBAY', 'HTTP_PROXY', 'APPLE_ICLOUD', 'APPLE', 'CLOUDFLARE', 'HTTP_CONNECT', 'WHATSAPP', 'IP_ICMP', 'HTTP_CONNECT', 'OFFICE_365', 'DROPBOX', 'OFFICE_365', 'WHATSAPP', 'CONTENT_FLASH', 'SPOTIFY', 'HTTP_PROXY', 'SSL', 'DNS', 'FACEBOOK', 'HTTP', 'REST_APP']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
0.22035395173794547


2020-04-27 16:59:05 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:59:05 INFO     Writing run file
2020-04-27 16:59:05 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:59:05 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:59:05 INFO     Running clustering...


Time: 478 sec.

2020-04-27 17:07:04 INFO     Extracting autoclass results
2020-04-27 17:07:04 INFO     Found 3200 cases classified in 32 classes
2020-04-27 17:07:07 INFO     Aggregating input data
2020-04-27 17:07:07 INFO     Writing classes + probabilities .tsv file


[0.07105263157894737, 0.0924092409240924, 0.09689922480620156, 0.09375, 0.22018348623853212, 0.11602209944751381, 0.14124293785310735, 0.1095890410958904, 0.09734513274336283, 0.08661417322834646, 0.3088235294117647, 0.1282051282051282, 0.10743801652892562, 1.0, 0.11538461538461539, 0.31645569620253167, 0.29333333333333333, 0.2318840579710145, 0.11764705882352941, 0.1388888888888889, 0.1875, 0.22413793103448276, 0.16216216216216217, 1.0, 0.17777777777777778, 0.15384615384615385, 0.35294117647058826, 0.25, 0.6666666666666666, 0.4444444444444444, 0.7142857142857143, 0.25]
['CLOUDFLARE', 'AMAZON', 'MS_ONE_DRIVE', 'INSTAGRAM', 'DROPBOX', 'HTTP', 'CONTENT_FLASH', 'SPOTIFY', 'GMAIL', 'MS_ONE_DRIVE', 'CONTENT_FLASH', 'DROPBOX', 'EBAY', 'IP_ICMP', 'APPLE_ITUNES', 'OFFICE_365', 'APPLE', 'DNS', 'SSL', 'EBAY', 'MSN', 'WHATSAPP', 'DNS', 'DNS', 'HTTP_CONNECT', 'HTTP', 'HTTP', 'DROPBOX', 'DNS', 'YAHOO', 'INSTAGRAM', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,